<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT20/blob/litneet64/Lab-3/litneet64_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 3: Iterative Methods**
**Pablo Aravena**

# **Abstract**

 We are tasked with coding the implementations for 3 non-direct, iterative methods to solve the linear equation $Ax = b$ and the root finding problem $f(x^*) = 0$.  These were the Jacobi iteration and the Gauss-Siedel method, and the Newton method, respectively.

# **About the code**

In [0]:
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2019 Pablo Aravena (pjan2@kth.se)

# Based on the template by Johan Hoffman (jhoffman@kth.se)
# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

In [0]:
# Load neccessary modules.
from google.colab import files

import numpy as np
import unittest
import random as rd
from sympy import diff
from sympy.abc import x as sp_x
from sympy.utilities.lambdify import lambdify
from math import isclose

# **Introduction**

We implemented 3 iterative methods to solve the linear system $Ax = b$, and the equation $f(x^*) = 0$. Some caution was taken at the creation of the tests for the matrix related methods, as these associated matrices need to possess some specific conditions for these methods to converge. Hence some grooming was necessary for these randomly generated matrices. 

For the newton method, a small error tolerance and a big number of iterations was put to it to ensure good approximations for the tests.

# **Methods**

#### Jacobi method
  Following what it's shown on the lecture notes on $7.3$ about the Jacobi Iteration, follows:

\begin{equation*}
    x_i^{(n+1)} = \frac{1}{a_{ii}} \cdot \left(b_i - \sum_{i \neq j} a_{ij}\cdot x_j^{(n)} \right)
\end{equation*}

So we get a better approximation element-wise after each iteration, which we limited as $5.000$ if a good enough approximation (error less than $10^{-9}$) is not found by then.

In [0]:
# method for the jacobi iteration
def jacobiIter(A, b):
    # iteration limit
    iter_limit = 5000
    
    # zero vector
    zero_vect = np.zeros((b.shape[0], b.shape[1]))
    
    # initial guess and iter counter
    new_x = x = np.zeros((b.shape[0], b.shape[1]))
    k = 0 
    
    # initial diff
    error = False
    
    # while we don't converge with a good approximation
    while not error and k < iter_limit:
        new_x = np.zeros((b.shape[0], b.shape[1]))
        
        # go through every element that's not in the diagonal
        for row in range(A.shape[0]):
            sum_1 = 0
            
            # ... and make an element by element mult if indexes aren't equal
            for col in range(A.shape[1]):
                if row != col:
                    sum_1 += A[row, col] * x[col, 0]
            
            # update value for x_n+1
            new_x[row, 0] = (1 / A[row, row]) * (b[row, 0] - sum_1)
        
        
        # increase iteration counter and update x for the next one
        k += 1
        x = new_x
        
        # re-calculate new error (gives out a boolean val)
        error = np.allclose(np.dot(A, x) - b, zero_vect)
    
    return x


#### Gauss-Siedel Method

This method is similar to the previous in both the error tolerance and iteration limit, except that the $A$ matrix is splitted into $2$ where one is the lower-triangular matrix and the other one the upper-triangular one (both diagonal exclusive). It follows then:

\begin{equation*}
    x_i^{(n+1)} = \frac{1}{a_{ii}} \cdot \left(b_i - \sum_{i \lt j} a_{ij}\cdot x_j^{(n)} - \sum_{i \gt j} a_{ij}\cdot x_j^{(n+1)} \right)
\end{equation*}

In [0]:
# gauss-siedel
def gaussSiedel(A, b):
    # iteration limit
    iter_limit = 5000
    
    # zero vector
    zero_vect = np.zeros((b.shape[0], b.shape[1]))
    
    # create initial guess and iter counter
    new_x = x = np.zeros((b.shape[0], b.shape[1]))
    k = 0
    
    # initial error
    error = False
    
    # run until iteration limit is reached or a good approximation is found
    while not error and k < iter_limit:
        for row in range(A.shape[0]):
            sum_1 = sum_2 = 0
        
            for col in range(A.shape[1]):
                # check for indexes for their corresponding sums
                if row < col:
                    sum_1 += A[row, col] * new_x[col, 0]
                elif row > col:
                    sum_2 += A[row, col] * x[col, 0]
            
            # update val on solution vector
            x[row, 0] = (1 / A[row, row]) * (b[row, 0] - sum_1 - sum_2)
        
        
            # re-calculate error and increase iter counter
            error = np.allclose(np.dot(A, x) - b, zero_vect)
            k += 1
        
    return x
        


#### Newton's method
  For this method we just used:
  
\begin{equation*}
    x^{(n+1)} = x^{(n)} - \frac{f(x^{(n)})}{f^{'}(x^{(n)})}
\end{equation*}

A limit of $100.000$ iterations was placed in case the approximation didn't reach a close enough value to the root within an error margin of $10^{-30}$.

In [0]:
# newton's method
def newtonMethod(func):
    # iteration limit
    iter_limit = 100_000
    
    # get the derivative of our function
    temp_diff = diff(func(sp_x), sp_x)
    d_func = lambdify(sp_x, temp_diff)
    
    # initial guess and iter counter
    x = new_x = 0
    k = 0
    
    while (abs(func(x)) > 1e-30) and k < iter_limit:
        x = new_x - func(new_x) / d_func(new_x)
        
        # update new x and increase iteration counter
        new_x = x
        k += 1
        
    return x


#### Tester class

We implemented a class inheriting from the `unittest` framework as to run some test cases at the end. These are bulk tests with randomized dimensions and elements for the matrices, and randomized coefficients for the cubic polynomials used to test the Newton Method. A high-order helper function was created as to help in reusing code for common patterns. The chosen number of test cases is $1.000$. 

For the tests that required matrices, the numpy methods `allclose` and `linalg.solve` were used to get the real solutions and also to compare them with our given ones. One important detail was to actually create an $A$ matrix that would make these methods converge. We know that a sufficient condition for both methods to converge is that the $A$ matrix used should be strictly diagonal dominant, that is if for every row:
 
\begin{equation*}
    |a_{ii}| > \sum_{i \neq j} |a_{ij}|
\end{equation*}

As we are just using positive real numbers for the elements of these matrices, the absolute value is not required in our `makeDiagonalDom` function.

To test the Newton method we just evaluated the given root of that polynomial inside it and compared it to $0$, as $f(x^*) = 0$, where $x^*$ is a root, and we used the method `isclose` for this, which has a tolerance for errors of $10^{-10}$.

In [0]:
# generic tester class
class method_tester(unittest.TestCase):
    # number of test cases and maximum number of rows/cols
    TEST_CASES = 1000
    MAX_ROWS = 10
    
    # maximum for polynomial coeffs
    MAX_COEFF = 30
    
    # make a matrix diagonal dominant
    def makeDiagonalDom(self, A):
        # go through every row
        for i in range(A.shape[0]):
            # get the current row and the sum of it's elements
            curr_row = A[i,]
            row_sum = sum(curr_row)
            
            # replace the diagonal element with a big enough one
            A[i,i] = row_sum + 2
    
    
    # general bulk tester for matrix methods
    def bulkHelper(self, matrix_method):
        # go through 1.000 test cases
        for i in range(self.TEST_CASES):
            # get random row and column dimensions (should be a square matrix)
            rows_cols = rd.randint(2, self.MAX_ROWS + 1)
            
            # get a random matrix and condition it to make these methods converge
            A = np.random.rand(rows_cols, rows_cols) * 2    
            self.makeDiagonalDom(A)
            
            # generate random vector (should be vertical for our functions)
            b = np.random.rand(rows_cols, 1) * 5
            
            # test the method with the corresponding real value given by numpy (error tolerance of 1e-05)
            self.assertTrue(np.allclose(matrix_method(A, b), np.linalg.solve(A, b)))
            
    
    # random bulk tester for the jacobi iteration method
    def testJacobi(self):
        self.bulkHelper(jacobiIter)
        
    # random bulk tester for the Gauss-Siedel iteration method
    def testGS(self):
        self.bulkHelper(gaussSiedel)
        
    # random bulk tester for Newton's method
    def testNewton(self):
        # go through 1.000 test cases 
        for i in range(self.TEST_CASES):
            # list holding the coefficients for our cubic polynomial
            coeffs = [0 for i in range(4)]
            
            for i in range(4):
                coeffs[i] = rd.randint(1, self.MAX_COEFF + 1)
                
            # generate cubic polynomial
            pol = lambda x: coeffs[0]*x**3 + coeffs[1]*x**2 + coeffs[2]*x - coeffs[3]
            
            root = newtonMethod(pol)
            
            # the function evaluated on it's own root should be 0 (f(x_0) == 0) (error tolerance of 1e-10)
            self.assertTrue(isclose(abs(pol(root)), 0, abs_tol = 1e-10))
        

# **Results**

if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

# **Discussion**

The results shows that no errors were caused by the tests, proving they work as intended. It's interesting to note that even though we needed to create randomized matrices, both dimension-wise and element-wise, this was not enough to ensure that our methods would converge, thus some modifications were needed on these matrices before being used for testing. 